In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install unidecode
!pip install langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=22d4d6a0142f9aa5415a8a71b93c7dda15b6064f3100e5efe344e9c05e85ca89
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import pickle
import json
#import redshift_connector
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)
import numpy as np
import unidecode
import re

from collections import Counter
from math import ceil
from langdetect import detect
from random import sample

In [1]:
rutaDatos = '../Datos'
ruta ='./'

## Exploring the ROR Data to Create Artificial Training Data

In [ ]:
# Data was downloaded from the ROR website for the date seen in the file string below
# -------> https://ror.readme.io/docs/data-dump
ror = pd.read_json(f"{rutaDatos}/v1.50-2024-07-29-ror-data.json")

In [ ]:
#Vale
ror = ror[ror.country.apply(lambda x: x['country_code'] == 'AR')]  #Me quedo sólo con las argentinas
print(ror.shape)
ror.head(2)

(388, 16)


,id,name,types,links,aliases,acronyms,status,wikipedia_url,labels,email_address,ip_addresses,established,country,relationships,addresses,external_ids
213,https://ror.org/00pt8r998,National University of the Littoral,[Education],[http://www.unl.edu.ar/],[],[UNL],active,http://en.wikipedia.org/wiki/National_University_of_the_Littoral,"[{'label': 'Universidad Nacional del Litoral', 'iso639': 'es'}, {'label': 'Université Nationale du Littoral', 'iso639': 'fr'}]",,[],1889.0,"{'country_code': 'AR', 'country_name': 'Argentina'}","[{'type': 'Child', 'label': 'Centro de Investigación de Métodos Computacionales', 'id': 'https://ror.org/0041aya12'}, {'type': 'Child', 'label': 'Instituto Nacional de Limnología', 'id': 'https://ror.org/042765079'}, {'type': 'Child', 'label': 'Instituto de Agrobiotecnología del Litoral', 'id': 'https://ror.org/057915t59'}, {'type': 'Child', 'label': 'Instituto de Desarrollo Tecnológico para la Industria Química', 'id': 'https://ror.org/01kwwh635'}, {'type': 'Child', 'label': 'Instituto de Física del Litoral', 'id': 'https://ror.org/058xqms97'}]","[{'line': None, 'lat': -31.634444, 'lng': -60.705278, 'postcode': None, 'primary': False, 'city': 'Santa Fe', 'state': None, 'state_code': None, 'country_geonames_id': 3865483, 'geonames_city': {'id': 3836277, 'city': 'Santa Fe', 'nuts_level1': {'code': None, 'name': None}, 'nuts_level2': {'code': None, 'name': None}, 'nuts_level3': {'code': None, 'name': None}, 'geonames_admin1': {'id': 3836276, 'name': 'Santa Fe', 'ascii_name': 'Santa Fe', 'code': 'AR.21'}, 'geonames_admin2': {'id': 3852292, 'name': 'Departamento de La Capital', 'ascii_name': 'Departamento de La Capital', 'code': 'AR.21.82063'}, 'license': {'attribution': 'Data from geonames.org under a CC-BY 3.0 license', 'license': 'http://creativecommons.org/licenses/by/3.0/'}}}]","{'ISNI': {'preferred': None, 'all': ['0000 0001 2172 9456']}, 'FundRef': {'preferred': None, 'all': ['501100005746']}, 'OrgRef': {'preferred': None, 'all': ['7447908']}, 'Wikidata': {'preferred': None, 'all': ['Q265866']}, 'GRID': {'preferred': 'grid.10798.37', 'all': 'grid.10798.37'}}"
215,https://ror.org/00htwgm11,National University of Salta,[Education],[http://www.unsa.edu.ar/],[],[UNSa],active,http://en.wikipedia.org/wiki/National_University_of_Salta,"[{'label': 'Universidad Nacional de Salta', 'iso639': 'es'}]",None,[],1972.0,"{'country_code': 'AR', 'country_name': 'Argentina'}","[{'type': 'Related', 'label': 'Centro Científico Tecnológico Salta - Jujuy', 'id': 'https://ror.org/04tpkqz22'}]","[{'line': None, 'lat': -24.727561, 'lng': -65.407887, 'postcode': None, 'primary': False, 'city': 'Salta', 'state': None, 'state_code': None, 'country_geonames_id': 3865483, 'geonames_city': {'id': 3838233, 'city': 'Salta', 'nuts_level1': {'code': None, 'name': None}, 'nuts_level2': {'code': None, 'name': None}, 'nuts_level3': {'code': None, 'name': None}, 'geonames_admin1': {'id': 3838231, 'name': 'Salta', 'ascii_name': 'Salta', 'code': 'AR.17'}, 'geonames_admin2': {'id': 3862749, 'name': 'Departamento Capital', 'ascii_name': 'Departamento Capital', 'code': 'AR.17.66028'}, 'license': {'attribution': 'Data from geonames.org under a CC-BY 3.0 license', 'license': 'http://creativecommons.org/licenses/by/3.0/'}}}]","{'ISNI': {'preferred': None, 'all': ['0000 0004 0490 9553']}, 'FundRef': {'preferred': '501100006681', 'all': ['501100006681', '501100006682']}, 'OrgRef': {'preferred': None, 'all': ['18658316']}, 'Wikidata': {'preferred': None, 'all': ['Q6979266']}, 'GRID': {'preferred': 'grid.10821.3a', 'all': 'grid.10821.3a'}}"


In [ ]:
ror['alias_len'] = ror['aliases'].apply(len)
ror['acronyms_len'] = ror['acronyms'].apply(len)
ror['labels_len'] = ror['labels'].apply(len)
ror['address_len'] = ror['addresses'].apply(len)
ror['address'] = ror['addresses'].apply(lambda x: x[0])
ror['ror_id'] = ror['id'].apply(lambda x: x.split("/")[-1])
ror['types'] = ror['types'].apply(lambda x: x[0])

In [ ]:
ror[ror['ror_id']=='00htwgm11']

,id,name,types,links,aliases,acronyms,status,wikipedia_url,labels,email_address,ip_addresses,established,country,relationships,addresses,external_ids,alias_len,acronyms_len,labels_len,address_len,address,ror_id
215,https://ror.org/00htwgm11,National University of Salta,Education,[http://www.unsa.edu.ar/],[],[UNSa],active,http://en.wikipedia.org/wiki/National_University_of_Salta,"[{'label': 'Universidad Nacional de Salta', 'iso639': 'es'}]",None,[],1972.0,"{'country_code': 'AR', 'country_name': 'Argentina'}","[{'type': 'Related', 'label': 'Centro Científico Tecnológico Salta - Jujuy', 'id': 'https://ror.org/04tpkqz22'}]","[{'line': None, 'lat': -24.727561, 'lng': -65.407887, 'postcode': None, 'primary': False, 'city': 'Salta', 'state': None, 'state_code': None, 'country_geonames_id': 3865483, 'geonames_city': {'id': 3838233, 'city': 'Salta', 'nuts_level1': {'code': None, 'name': None}, 'nuts_level2': {'code': None, 'name': None}, 'nuts_level3': {'code': None, 'name': None}, 'geonames_admin1': {'id': 3838231, 'name': 'Salta', 'ascii_name': 'Salta', 'code': 'AR.17'}, 'geonames_admin2': {'id': 3862749, 'name': 'Departamento Capital', 'ascii_name': 'Departamento Capital', 'code': 'AR.17.66028'}, 'license': {'attribution': 'Data from geonames.org under a CC-BY 3.0 license', 'license': 'http://creativecommons.org/licenses/by/3.0/'}}}]","{'ISNI': {'preferred': None, 'all': ['0000 0004 0490 9553']}, 'FundRef': {'preferred': '501100006681', 'all': ['501100006681', '501100006682']}, 'OrgRef': {'preferred': None, 'all': ['18658316']}, 'Wikidata': {'preferred': None, 'all': ['Q6979266']}, 'GRID': {'preferred': 'grid.10821.3a', 'all': 'grid.10821.3a'}}",0,1,1,1,"{'line': None, 'lat': -24.727561, 'lng': -65.407887, 'postcode': None, 'primary': False, 'city': 'Salta', 'state': None, 'state_code': None, 'country_geonames_id': 3865483, 'geonames_city': {'id': 3838233, 'city': 'Salta', 'nuts_level1': {'code': None, 'name': None}, 'nuts_level2': {'code': None, 'name': None}, 'nuts_level3': {'code': None, 'name': None}, 'geonames_admin1': {'id': 3838231, 'name': 'Salta', 'ascii_name': 'Salta', 'code': 'AR.17'}, 'geonames_admin2': {'id': 3862749, 'name': 'Departamento Capital', 'ascii_name': 'Departamento Capital', 'code': 'AR.17.66028'}, 'license': {'attribution': 'Data from geonames.org under a CC-BY 3.0 license', 'license': 'http://creativecommons.org/licenses/by/3.0/'}}}",00htwgm11


In [ ]:
#Vale
import requests
resultadoP1 = requests.get('https://api.openalex.org/institutions?per-page=200&page=1&filter=country_code:ar')
resultadoP2 = requests.get('https://api.openalex.org/institutions?per-page=200&page=2&filter=country_code:ar')
with open(f"{ruta}/institOA-AR-P1.txt","w") as archivo:
    archivo.write(resultadoP1.text)
with open(f"{ruta}/institOA-AR-P2.txt","w") as archivo:
    archivo.write(resultadoP2.text)


In [ ]:
#Vale
with open(f"{rutaDatos}/institOA-AR-P1.txt","r") as archivo1:
    instText1 =archivo1.read()
with open(f"{rutaDatos}/institOA-AR-P2.txt","r") as archivo2:
    instText2 =archivo2.read()

instDict1 = json.loads(instText1)
instDict2 = json.loads(instText2)

instList1 = instDict1['results']
instList2 = instDict2['results']

instTotal = instList1 + instList2
instDF = pd.DataFrame.from_dict(instTotal)
instDF = instDF.rename(columns={"id": "affiliation_id", "ror": "ror_id"})
instDF.to_parquet(f"{rutaDatos}/OA_static_institutions_single_file.parquet")

instDF.sample()

In [ ]:
# this file is not provided but the needed data is all institutions in OpenAlex
# with the following columns: 'ror_id','affiliation_id'
insts = pd.read_parquet(f"{rutaDatos}/OA_static_institutions_single_file.parquet",columns=['affiliation_id','ror_id'])

In [ ]:
insts['ror_id'] = insts['ror_id'].apply(lambda x: x.split("/")[-1])
insts['raffiliation_id'] = insts['affiliation_id'].apply(lambda x: x.split("/")[-1])
insts.head(2)

,affiliation_id,ror_id
0,https://openalex.org/I151201029,03cqe8w59
1,https://openalex.org/I24354313,0081fs513


In [ ]:
ror_to_join = ror[['ror_id','name','aliases','acronyms','labels','country','types',
                   'address','alias_len','acronyms_len','labels_len','address_len']].copy()

In [ ]:
def get_geoname_admin(address_dict):
    try:
        geoname_admin = address_dict['geonames_city']['geonames_admin1']['name']
    except:
        geoname_admin = "None"

    return geoname_admin

In [ ]:
ror_to_join['country_code'] = ror_to_join['country'].apply(lambda x: x['country_code'])
ror_to_join['country_name'] = ror_to_join['country'].apply(lambda x: x['country_name'])
ror_to_join['city'] = ror_to_join['address'].apply(lambda x: x['city'])
ror_to_join['state'] = ror_to_join['address'].apply(lambda x: x['state'])
ror_to_join['region'] = ror_to_join['address'].apply(get_geoname_admin)
ror_to_join['institution'] = ror_to_join['name']

##### Looking at ROR Labels

In [ ]:
codes_to_ignore = ['ja','fa','hi','ko','bn','zh','ml','ru','el','kn','gu','mk','ne','te','hy',
                   'km','ti','kk','th','my','uk','pa','bg','ur','vi','ar','sr','he','ta','ka',
                   'am','mr','lo','mn','be','or','ba','si','ky','uz']

In [ ]:
labels = ror_to_join['labels'].explode().dropna().reset_index()
labels['label'] = labels['labels'].apply(lambda x: x['label'])
labels['iso639'] = labels['labels'].apply(lambda x: x['iso639'])

In [ ]:
labels[~labels['iso639'].isin(codes_to_ignore)].sample(20)

,index,labels,label,iso639
91,96398,"{'label': 'Centro de Investigaciones en Química Biológica de Córdoba', 'iso639': 'es'}",Centro de Investigaciones en Química Biológica de Córdoba,es
71,88692,"{'label': 'Interamerican Open University', 'iso639': 'en'}",Interamerican Open University,en
69,86982,"{'label': 'Université nationale de tucumán', 'iso639': 'fr'}",Université nationale de tucumán,fr
45,32270,"{'label': 'Universidad Nacional de Tierra del Fuego, Antártida e Islas del Atlántico Sur', 'iso639': 'es'}","Universidad Nacional de Tierra del Fuego, Antártida e Islas del Atlántico Sur",es
30,26102,"{'label': 'Universidad de la Cuenca del Plata', 'iso639': 'es'}",Universidad de la Cuenca del Plata,es
120,102076,"{'label': 'Conselho Latino-americano de Ciências Sociais', 'iso639': 'pt'}",Conselho Latino-americano de Ciências Sociais,pt
147,110078,"{'label': 'Universidade Nacional de Córdoba', 'iso639': 'pt'}",Universidade Nacional de Córdoba,pt
26,26087,"{'label': 'Université du centre d'études macroeconomiques d'argentine', 'iso639': 'fr'}",Université du centre d'études macroeconomiques d'argentine,fr
122,103661,"{'label': 'Santa Fe Science, Technology and Innovation Agency', 'iso639': 'en'}","Santa Fe Science, Technology and Innovation Agency",en
113,99199,"{'label': 'Comisión Nacional de Actividades Espaciales', 'iso639': 'es'}",Comisión Nacional de Actividades Espaciales,es


#### Getting string beginnings

Looking to introduce more variety into the artificial strings so that they include some header information such as "College of .." or "Department of...".

In [ ]:
with open(f"{ruta}/ror_string_beginnings/Company", 'r') as f:
    company_begs = f.readlines()

company_begs = list(set([x.rstrip('\n') for x in company_begs]))

In [ ]:
with open(f"{ruta}/ror_string_beginnings/Education_dept", 'r') as f:
    education_dept_begs = f.readlines()

education_dept_begs = list(set([x.rstrip('\n') for x in education_dept_begs]))

In [ ]:
with open(f"{ruta}/ror_string_beginnings/Education_college", 'r') as f:
    education_col_begs = f.readlines()

education_col_begs = list(set([x.rstrip('\n') for x in education_col_begs]))

In [ ]:
with open(f"{ruta}/ror_string_beginnings/Education_school", 'r') as f:
    education_school_begs = f.readlines()

education_school_begs = list(set([x.rstrip('\n') for x in education_school_begs]))

In [ ]:
with open(f"{ruta}/ror_string_beginnings/Healthcare", 'r') as f:
    healthcare_begs = f.readlines()

healthcare_begs = list(set([x.rstrip('\n') for x in healthcare_begs]))

In [ ]:
all_education = []
for beg in education_col_begs:
    all_education.append(f"College of {beg}")

for beg in education_dept_begs:
    all_education.append(f"Department of {beg}")
    all_education.append(f"Dep of {beg}")
    all_education.append(f"Dept of {beg}")
    all_education.append(f"Dep. of {beg}")
    all_education.append(f"Dept. of {beg}")

for beg in education_school_begs:
    all_education.append(f"School of {beg}")
    all_education.append(f"Sch. of {beg}")
    all_education.append(f"Sch of {beg}")
len(all_education)

272

In [ ]:
all_company = company_begs

In [ ]:
all_healthcare = []
for beg in healthcare_begs:
    all_healthcare.append(f"Department of {beg}")
    all_healthcare.append(f"Dep of {beg}")
    all_healthcare.append(f"Dept of {beg}")
    all_healthcare.append(f"Dep. of {beg}")
    all_healthcare.append(f"Dept. of {beg}")

##### Creating the artificial affiliation strings

We would like to take advantage of the ROR data and use it to supplement/augment the current affiliation string data in OpenAlex. This could potentially allow for the Institution Tagger model to predict on institutions that have not yet had affiliation strings added to OpenAlex.

In [ ]:
type_preinst_dict = {'Company': all_company,
                     'Education': all_education,
                     'Healthcare': all_healthcare}

In [ ]:
def create_affiliation_string_from_scratch(institution, city, state, country, region, aliases, labels,
                                           acronyms, inst_type):
    aff_strings = []
    if aliases:
        aliases = [institution] + aliases
    else:
        aliases = [institution]
    if labels:
        for label in labels:
            if label['iso639'] in codes_to_ignore:
                pass
            else:
                aliases.append(label['label'])
    if acronyms:
        for acronym in acronyms:
            aliases.append(acronym)
    for alias in aliases:
        if "r&d" not in alias.lower():
            alias = alias.replace(" & ", " and ")
        match_string = unidecode.unidecode(alias)
        match_string = match_string.lower().replace(" ", "")
        match_string = "".join(re.findall("[a-zA-Z0-9]+", match_string))
        alias = unidecode.unidecode(alias)
        if ((state is None) & (region != city) & (city is not None) &
            (country is not None) & (region is not None)):
            region = unidecode.unidecode(region)
            country = unidecode.unidecode(country)
            city = unidecode.unidecode(city)
            aff_strings.append([f"{alias}, {city}, {region}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}, {region}", match_string])
            aff_strings.append([f"{alias}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}", match_string])
            aff_strings.append([f"{alias}, {region}", match_string])
            aff_strings.append([f"{alias} {city} {region} {country}", match_string])
            aff_strings.append([f"{alias} {city} {country}", match_string])
            aff_strings.append([f"{alias} {city} {region}", match_string])
            aff_strings.append([f"{alias} {country}", match_string])
            aff_strings.append([f"{alias} {city}", match_string])
            aff_strings.append([f"{alias} {region}", match_string])
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(5):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                else:
                    for i in range(2):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])

        elif (state is not None) & (city is not None) & (country is not None):
            state = unidecode.unidecode(state)
            country = unidecode.unidecode(country)
            city = unidecode.unidecode(city)
            aff_strings.append([f"{alias}, {city}, {state}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}, {state}", match_string])
            aff_strings.append([f"{alias}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}", match_string])
            aff_strings.append([f"{alias}, {state}", match_string])
            aff_strings.append([f"{alias} {city} {state} {country}", match_string])
            aff_strings.append([f"{alias} {city} {country}", match_string])
            aff_strings.append([f"{alias} {city} {state}", match_string])
            aff_strings.append([f"{alias} {country}", match_string])
            aff_strings.append([f"{alias} {city}", match_string])
            aff_strings.append([f"{alias} {state}", match_string])
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                else:
                    for i in range(1):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
        elif (city is not None) & (country is not None):
            country = unidecode.unidecode(country)
            city = unidecode.unidecode(city)
            aff_strings.append([f"{alias}, {city}, {country}", match_string])
            aff_strings.append([f"{alias}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}", match_string])
            aff_strings.append([f"{alias} {city} {country}", match_string])
            aff_strings.append([f"{alias} {country}", match_string])
            aff_strings.append([f"{alias} {city}", match_string])
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                else:
                    for i in range(1):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
        elif (country is not None):
            country = unidecode.unidecode(country)
            aff_strings.append([f"{alias}, {country}", match_string])
            aff_strings.append([f"{alias} {country}", match_string])
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(4):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])

                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                else:
                    for i in range(1):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])

        else:
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(5):
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                else:
                    for i in range(2):
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
    return aff_strings

In [ ]:
def get_institutions_list(institution, aliases, labels, acronyms):
    aff_strings = []
    if aliases:
        aliases = [institution] + aliases
    else:
        aliases = [institution]
    if labels:
        for label in labels:
            if label['iso639'] in codes_to_ignore:
                pass
            else:
                aliases.append(label['label'])
    return aliases

In [ ]:
ror_to_join['affs'] = ror_to_join \
.apply(lambda x: get_institutions_list(x.institution, x.aliases,
                                       x.labels, x.acronyms), axis=1)
ror_to_join['aff_string'] = ror_to_join \
.apply(lambda x: create_affiliation_string_from_scratch(x.institution, x.city, x.state,
                                                        x.country_name, x.region,
                                                        x.aliases, x.labels, x.acronyms, x.types), axis=1)
ror_to_join['aff_string_len'] = ror_to_join['aff_string'].apply(len)
ror_to_join_final = ror_to_join.explode("aff_string").copy()

##### Looking to quickly get combinations of city/region/country

In [ ]:
art_empty_affs = ror_to_join[['city','region','country_name']].dropna().copy()

In [ ]:
art_empty_affs.shape

(347, 3)

In [ ]:
art_empty_affs.sample(10)

,city,region,country_name
102219,Buenos Aires,Buenos Aires F.D.,Argentina
59582,Rosario,Santa Fe,Argentina
98272,Buenos Aires,Buenos Aires F.D.,Argentina
74004,Mar del Plata,Buenos Aires,Argentina
97663,Rafaela,Santa Fe,Argentina
98400,Buenos Aires,Buenos Aires F.D.,Argentina
51603,San Luis,San Luis,Argentina
74925,Buenos Aires,Buenos Aires F.D.,Argentina
98393,Buenos Aires,Buenos Aires F.D.,Argentina
100073,Buenos Aires,Buenos Aires F.D.,Argentina


In [ ]:
art_empty_affs['original_affiliation_1'] = art_empty_affs.apply(lambda x: f"{x.city}, {x.country_name}", axis=1)

In [ ]:
art_empty_affs['original_affiliation_2'] = art_empty_affs.apply(lambda x: f"{x.city}, {x.region}, {x.country_name}", axis=1)

In [ ]:
city_country = art_empty_affs.sample(347).drop_duplicates()['original_affiliation_1'].to_list()

In [ ]:
city_region_country = art_empty_affs.sample(347).drop_duplicates()['original_affiliation_2'].to_list()

Some of these string will be used to train the model that only seeing a city/region/country should be a "no prediction"

In [ ]:
artificial_empty_affs_DF = pd.DataFrame(zip(city_country+city_region_country),
             columns=['original_affiliation']) \
    .drop_duplicates(subset=['original_affiliation'])#\
#    .to_parquet(f"{ruta}/artificial_empty_affs.parquet")
artificial_empty_affs_DF.to_parquet(f"{ruta}/artificial_empty_affs.parquet")

artificial_empty_affs_DF.shape

(120, 1)

In [ ]:
ror_to_join_final = ror_to_join[['ror_id','aff_string']].explode("aff_string").copy()
ror_to_join_final.shape

(51430, 2)

In [ ]:
ror_to_join_final['original_affiliation'] = ror_to_join_final['aff_string'].apply(lambda x: x[0])

In [ ]:
print(ror_to_join_final.head(5))
insts.head(5)

        ror_id  \
213  00pt8r998   
213  00pt8r998   
213  00pt8r998   
213  00pt8r998   
213  00pt8r998   

                                                                                      aff_string  \
213  [National University of the Littoral, Santa Fe, Argentina, nationaluniversityofthelittoral]   
213            [National University of the Littoral, Argentina, nationaluniversityofthelittoral]   
213             [National University of the Littoral, Santa Fe, nationaluniversityofthelittoral]   
213    [National University of the Littoral Santa Fe Argentina, nationaluniversityofthelittoral]   
213             [National University of the Littoral Argentina, nationaluniversityofthelittoral]   

                                         original_affiliation  
213  National University of the Littoral, Santa Fe, Argentina  
213            National University of the Littoral, Argentina  
213             National University of the Littoral, Santa Fe  
213    National University of the 

,affiliation_id,ror_id
0,https://openalex.org/I151201029,03cqe8w59
1,https://openalex.org/I24354313,0081fs513
2,https://openalex.org/I874386039,01tjs6929
3,https://openalex.org/I166401450,056tb7j80
4,https://openalex.org/I53241121,05rxmkq09


The rest of the artificial strings are exported so that they can be combined with the historical affiliation data to create the final training data set.

In [ ]:
ror_strings_Merge = ror_to_join_final.merge(insts, how='inner', on='ror_id')[['original_affiliation','affiliation_id']]

ror_strings_Merge.shape

(45981, 2)

In [ ]:
ror_to_join_final.merge(insts, how='inner',
                        on='ror_id')[['original_affiliation','affiliation_id']] \
.to_parquet(f"{rutaDatos}/ror_strings.parquet")